## Neurohack - Challenge 2 London team e
### Author: Maitreyee Wairagkar
#### Last update: 13/01/2022

## Load data

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d

'''
# Convert .dat to csv
data = pd.io.stata.read_stata('H_DAD_w1a3.dta')
data.to_csv('H_DAD_w1a3.csv')
'''

data = pd.read_csv('H_DAD_w1a3.csv')

## Pre-processing by removing non-relevant features and NaN columns

In [ ]:
remove_features = ['prim_key','hhid','pnc','pn','r1stateid','r1iwy_d','r1iwm_d','r1lasidy','rabyear', 
                   'rabmonth', 'r1lang_d', 'r1location', 'r1mheight', 'r1mweight', 'r1ht_flag', 
                   'r1wt_flag', 'r1htlimbs', 'r1midarm', 'r1calf', 'r1kneeht', 'r1raterid1', 
                   'r1raterid2', 'r1raterid3', 'r1phase', 'r1iwstat_d', 'r1inf_age', 'r1inf_gendr', 
                   'r1inf_educ', 'r1inf_rel', 'r1inf_freq', 'r1inf_care', 'r1inf_yrs', 'r1mbmi', 'r1bmicat',] 

potential_labels = ['r1cdr_final','r1cdr_incon','r1cdr_mem1','r1cdr_ori1','r1cdr_jud1','r1cdr_com1',
                    'r1cdr_hom1','r1cdr_per1','r1cdr_scor1','r1cdr_mem2','r1cdr_ori2','r1cdr_jud2',
                    'r1cdr_com2','r1cdr_hom2','r1cdr_per2','r1cdr_scor2','r1cdr_mem3','r1cdr_ori3',
                    'r1cdr_jud3','r1cdr_com3','r1cdr_hom3','r1cdr_per3','r1cdr_scor3']

In [ ]:
# Pre-processing: remove non-important features

# remove rows with missing values for clinician evaluation (this discards phase 1)
df = data.dropna(subset = ['r1cdr_final'])

labels = df['r1cdr_final']
gender = df['ragender']

df = df.drop(columns=potential_labels)
df = df.drop(columns=remove_features)

# remove columns with NaN
df = df.dropna(axis=1)

df.shape

## T-SNE Visualisation of Features

In [ ]:
# Compute and plot T-SNE
from sklearn.manifold import TSNE

# We want to get TSNE embedding with 2 dimensions
n_components = 3
tsne = TSNE(n_components)
tsne_result = tsne.fit_transform(df)
tsne_result.shape

# Plot T-SNE
scatter_x = np.array(tsne_result[:,0])
scatter_y = np.array(tsne_result[:,1])
scatter_z = np.array(tsne_result[:,2])
group = np.array(labels)
cdict = {0: 'green', 0.5: 'blue', 1:'orange', 2:'red', 3: 'purple'}

fig = plt.figure(figsize = (7, 7))
ax = plt.axes(projection ="3d")

for g in np.unique(group):
    ix = np.where(group == g)
    ax.scatter3D(scatter_x[ix], scatter_y[ix], scatter_z[ix], c = cdict[g], label = g)
ax.legend()
plt.show()

## Histogram of pre-processed dataset

In [ ]:
# Histogram
# Creating dataset
a = []
for i in labels:
    if i == 0: # no dementia
        a.append(i)
    if i == 0.5: # questionable
        a.append(1)
    if i == 1: # mild
        a.append(2)
    if i == 2: # moderate
        a.append(3)
    if i == 3: # severe
        a.append(4)

# Creating histogram
labl, counts = np.unique(a, return_counts=True)

fig = plt.figure(figsize = (9, 6))
plt.rc('font', size=15) 

plt.bar(labl, counts, align='center')
plt.gca().set_xticks(labl)
plt.gca().set_xticklabels(('No\nimpairment', 'Questionable\nimpairment', 'Mild\ndementia', 'Moderate\ndementia', 'Severe\ndementia'))
plt.xlabel('\nConsensus Clinical Dementia Rating Score')
plt.ylabel('Number of people')
plt.show()


## PCA on cleaned data (all features)

In [ ]:
# standardise the data before PCA
from sklearn.preprocessing import MinMaxScaler

x = MinMaxScaler().fit_transform(df) # normalizing the features
x.shape


# PCA on all features
from sklearn.decomposition import PCA

pca = PCA() #n_components=3
principalComponents = pca.fit_transform(x)

plt.plot(pca.explained_variance_ratio_)

In [303]:
# plot components

#%matplotlib qt

comp =[0,2,4] # select components to plot
scatter_x = np.array(principalComponents[:,comp[0]])
scatter_y = np.array(principalComponents[:,comp[1]])
scatter_z = np.array(principalComponents[:,comp[2]])
group = np.array(labels)
cdict = {0: 'green', 0.5: 'blue', 1:'orange', 2:'red', 3: 'purple'}
label_names = {0:'No impairment', 0.5:'Questionable impairment', 1:'Mild dementia', 2:'Moderate dementia', 3:'Severe dementia'}

fig = plt.figure(figsize = (7, 7))
plt.rc('font', size=10) 
ax = plt.axes(projection ="3d")


for g in np.unique(group):
    ix = np.where(group == g)
    ax.scatter(scatter_x[ix], scatter_y[ix], scatter_z[ix], c = cdict[g], label = label_names[g] )
    
ax.legend(bbox_to_anchor=(0, 1.15),loc='upper left', title='Clinical Dementia Rating')
plt.xlabel('Principal Component '+str(comp[0]+1))
plt.ylabel('Principal Component '+str(comp[1]+1))
ax.set_zlabel('Principal Component '+str(comp[2]+1))

#ax.view_init(0, -190)

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

plt.show()

In [ ]:
# Write PCA output in csv
pc_d=pd.DataFrame(principalComponents)
pc_d.to_csv('principalComponents.csv')

l=pd.DataFrame({'CDR_score':labels,
                    'gender': gender})
l.to_csv('data_labels.csv')

p=pd.DataFrame(pca.components_)
p.to_csv('PCAweights.csv')

v=pd.DataFrame(pca.explained_variance_ratio_)
v.to_csv('PCAvariance.csv')

col = pd.DataFrame(df.columns)
col.to_csv('PCAcolumns.csv')

## Multiclass classification using SVM (all features) 10-fold cross-validation

In [ ]:
import numpy.matlib
import random
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn import svm
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report


# 1. prepare data 
group = np.array(labels)

# get indices for different classes
for g in np.unique(group):
    ix = np.where(group == g)
    if g == 0:
        class_1 = ix
    if g == 0.5: 
        class_2 = ix
    if g == 1:
        class_3 = ix
    if g == 2: 
        class_4 = ix
    if g == 3:
        class_5 = ix
        
# Get data for each class and add class label in the last column
dat_1 = np.concatenate((np.squeeze(principalComponents[class_1,:]),np.ones([len(np.squeeze(class_1)),1])),axis=1)
dat_2 = np.concatenate((np.squeeze(principalComponents[class_2,:]),np.ones([len(np.squeeze(class_2)),1])*2),axis=1)
dat_3 = np.concatenate((np.squeeze(principalComponents[class_3,:]),np.ones([len(np.squeeze(class_3)),1])*3),axis=1)
dat_4 = np.concatenate((np.squeeze(principalComponents[class_4,:]),np.ones([len(np.squeeze(class_4)),1])*4),axis=1)
dat_5 = np.concatenate((np.squeeze(principalComponents[class_5,:]),np.ones([len(np.squeeze(class_5)),1])*5),axis=1)

# super sampling: repeat matrices to match number of samples in largest class
dat_1 = np.matlib.repmat(dat_1,int(np.round(dat_2.shape[0]/dat_1.shape[0])),1)
dat_3 = np.matlib.repmat(dat_3,int(np.round(dat_2.shape[0]/dat_3.shape[0])),1)
dat_4 = np.matlib.repmat(dat_4,int(np.round(dat_2.shape[0]/dat_4.shape[0])),1)
dat_5 = np.matlib.repmat(dat_5,int(np.round(dat_2.shape[0]/dat_5.shape[0])),1)

# collect dat from all classes together and shuffle the order
all_dat = np.concatenate((dat_1,dat_2,dat_3,dat_4,dat_5),axis=0)
random.shuffle(all_dat)

# train-test split
#X_train, X_test, y_train, y_test = train_test_split(all_dat[:,0:-1], all_dat[:,-1], test_size=0.2)

# Get splits for Kfold cross-validation
kf = KFold(n_splits = 10, shuffle = True, random_state = 124)

In [ ]:
# 2. Classification with SVM

# Define the model
clf = svm.SVC(kernel='rbf', C=0.5) # C is regularisation

# Cross-validation and training

X_skf = all_dat[:,0:-1] # data
Y_skf = all_dat[:,-1]   # labels

# Normalise data
X_skf = MinMaxScaler().fit_transform(X_skf) # normalizing the features

fold_no = 0
cm =np.zeros([5,5]) # initialise empty confusion matrix
classi_report = []
train_score = []
test_score = []
loss = []
all_y_true = []
all_y_pred = []
cnt = 0

for train_idx, test_idx in skf.split(X_skf, Y_skf): 
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} for SVM ...')
    # Fit the model
    clf.fit(X_skf[train_idx,:], Y_skf[train_idx])

    # Make Prediction
    y_pred = clf.predict(X_skf[test_idx,:])
    
    # Making the Confusion Matrix and record performance
    cm = cm + confusion_matrix(Y_skf[test_idx],y_pred)
    classi_report.append(classification_report(Y_skf[test_idx],y_pred))
    
    train_score.append(clf.score(X_skf[train_idx,:], Y_skf[train_idx]))
    test_score.append(clf.score(X_skf[test_idx,:], Y_skf[test_idx]))
    loss.append(metrics.mean_squared_error(Y_skf[test_idx], y_pred))
    
    all_y_true.append(Y_skf[test_idx])
    all_y_pred.append(y_pred)
    
    print(cm)
    print("\n")
    print(classi_report[cnt])

    print("Training set score for SVM: %f" %train_score[cnt] )
    print("Testing  set score for SVM: %f" %test_score[cnt] )
    print("Loss per fold: %f" %loss[cnt])
    cnt=cnt+1
  
    fold_no+=1



In [302]:
# Print results 

CM = confusion_matrix(np.hstack(all_y_true),np.hstack(all_y_pred),normalize='true')*100
CM = np.round(CM,2)
print("Confusion matrix for multiclass classifier (%):\n")
print("                No dementia  "+ 'Questionable  '+ 'Mild   '+ 'Moderate  '+ 'Severe  ')
print("No dementia     "+ str(CM[0,0]) + "        " + str(CM[0,1])+ "          " + str(CM[0,2])+ "    " + str(CM[0,3])+ "       " + str(CM[0,4]))
print("Questionable    "+ str(CM[1,0]) + "        " + str(CM[1,1])+ "         " + str(CM[1,2])+ "    " + str(CM[1,3])+ "       " + str(CM[1,4]))
print("Mild            "+ str(CM[2,0]) + "          " + str(CM[2,1])+ "          " + str(CM[2,2])+ "  " + str(CM[2,3])+ "       " + str(CM[2,4]))
print("Moderate        "+ str(CM[3,0]) + "          " + str(CM[3,1])+ "           " + str(CM[3,2])+ "    " + str(CM[3,3])+ "     " + str(CM[3,4]))
print("Severe.         "+ str(CM[4,0]) + "          " + str(CM[4,1])+ "           " + str(CM[4,2])+ "    " + str(CM[4,3])+ "       " + str(CM[4,4]))

print('--------------------------------------------------------------------')
print()
print('Model Performance: ')
print()
report = classification_report(np.hstack(all_y_true),np.hstack(all_y_pred), output_dict=True)
print("Class                       Precision       Recall      f1-score")
print("No dementia                 "+ str(np.round(report['1.0']['precision']*100,2))+"           "+
                                      str(np.round(report['1.0']['recall']*100,2)) +"       "+
                                      str(np.round(report['1.0']['f1-score']*100,2)))
print("Questionable impairment     "+ str(np.round(report['2.0']['precision']*100,2))+"           "+
                                      str(np.round(report['2.0']['recall']*100,2)) +"       "+
                                      str(np.round(report['2.0']['f1-score']*100,2)))
print("Mild dementia               "+ str(np.round(report['3.0']['precision']*100,2))+"           "+
                                      str(np.round(report['3.0']['recall']*100,2)) +"       "+
                                      str(np.round(report['3.0']['f1-score']*100,2)))
print("Moderate dementia           "+ str(np.round(report['4.0']['precision']*100,2))+"           "+
                                      str(np.round(report['4.0']['recall']*100,2)) +"       "+
                                      str(np.round(report['4.0']['f1-score']*100,2)))
print("Severe dementia             "+ str(np.round(report['5.0']['precision']*100,2))+"           "+
                                      str(np.round(report['5.0']['recall']*100,2)) +"       "+
                                      str(np.round(report['5.0']['f1-score']*100,2)))
print()
print("Accuracy on test data: %0.2f" %np.round(np.mean(test_score)*100,2)+'%')
print("Accuracy on training data: %0.2f" %np.round(np.mean(train_score)*100,2)+'%')
print()
print("RMSE Loss: %0.3f" %np.mean(loss))



Confusion matrix for multiclass classifier (%):

                No dementia  Questionable  Mild   Moderate  Severe  
No dementia     99.61        0.39          0.0    0.0       0.0
Questionable    13.98        85.32         0.7    0.0       0.0
Mild            0.0          1.44          98.56  0.0       0.0
Moderate        0.0          0.0           0.0    100.0     0.0
Severe.         0.0          0.0           0.0    0.0       100.0
--------------------------------------------------------------------

Model Performance: 

Class                       Precision       Recall      f1-score
No dementia                 93.94           99.61       96.69
Questionable impairment     97.97           85.32       91.21
Mild dementia               98.89           98.56       98.73
Moderate dementia           100.0           100.0       100.0
Severe dementia             100.0           100.0       100.0

Accuracy on test data: 96.10%
Accuracy on training data: 98.41%

RMSE Loss: 0.039
